In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14157145719812891442
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2680591577788723834
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [9]:
!unzip -q Dataset.zip -d /content

In [10]:
# Preprocessing the Training set

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        '/content/Dataset/training_set',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


In [11]:
# Preprocessing the Test set

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        '/content/Dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


In [12]:
# Building the CNN

cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Training the CNN

cnn.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

cnn.fit(training_set,
        validation_data = test_set,
        epochs=20)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 95ms/step - accuracy: 0.5396 - loss: 0.6912 - val_accuracy: 0.7010 - val_loss: 0.6097
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 88ms/step - accuracy: 0.6795 - loss: 0.6060 - val_accuracy: 0.7230 - val_loss: 0.5624
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 89ms/step - accuracy: 0.7179 - loss: 0.5586 - val_accuracy: 0.7400 - val_loss: 0.5417
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.7351 - loss: 0.5277 - val_accuracy: 0.7530 - val_loss: 0.5178
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 43s 96ms/step - accuracy: 0.7538 - loss: 0.5066 - val_accuracy: 0.7595 - val_loss: 0.4939
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 85ms/step - accuracy: 0.7506 - loss: 0.4876 - val_accuracy: 0.7390 - val_loss: 0.5294
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 88ms/step - accuracy: 0.7636 - loss: 0.4821 - val_accuracy: 0.7665 - val_loss: 0.4976
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 90ms/step - accuracy: 0.7806 - loss: 0.4608 - val_accurac

In [15]:
# Saving our model

cnn.save("simple_cnn_20Epochs.h5")

In [17]:
# Loading our model

from tensorflow.keras.models import load_model

new_cnn = load_model("simple_cnn_20Epochs.h5")

In [19]:
# Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/Dataset/single_prediction/cat_or_dog_1.jpg', target_size= (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image/255.0)
if result[0][0] > 0.5:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step


In [21]:
print(prediction)

dog
